In [1]:
import ROOT

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x81c7020


In [2]:
%jsroot on

In [3]:
ROOT.EnableImplicitMT(6)
ROOT.TH1.SetDefaultSumw2()

In [4]:
ROOT.ildStyle.SetOptStat(0)
ROOT.ildStyle.SetPalette(ROOT.kBird)
ROOT.ildStyle.SetNumberContours(256)

In [5]:
ROOT.gInterpreter.Declare("#include <WWTools.h>")
ROOT.gInterpreter.Declare("#include <podio/GenericParameters.h>")

True

In [6]:
%%cpp
using namespace ROOT::VecOps;

In [7]:
tree = ROOT.TTree()
tree.AddFriend("events", "data/paris/nano/all.root")
tree.AddFriend("events", "data/paris/nano/all_params.root")

df = ROOT.RDataFrame(tree)

n_bins = 10
x_angle = 0.014 # rad

In [8]:
df.Describe()

Dataframe from TTree (in-memory)
with friends
  events data/paris/nano/all.root
  events data/paris/nano/all_params.root

Property                Value
--------                -----
Columns in total          104
Columns from defines        0
Event loops run             0
Processing slots            6

Column                                          Type                                                            Origin
------                                          ----                                                            ------
events.Parameters                               podio::GenericParameters                                        Dataset
events.Parameters._doubleMap                    map<string,vector<double> >                                     Dataset
events.Parameters._doubleMap.first              ROOT::VecOps::RVec<string>                                      Dataset
events.Parameters._doubleMap.second             ROOT::VecOps::RVec<vector<double>>                 

In [9]:
# df = df.Vary("nu_lvec", "ROOT::Math::PxPyPzMVector tmp(E_miss_lvec); return RVec<ROOT::Math::PxPyPzMVector>{tmp};", ["recoable_nu"], "nu_variation")
# df = df.Vary(("nu_lvec", "l_lvec", "hadronic_lvec"), """
#               ROOT::Math::PxPyPzMVector e_miss(E_miss_lvec);
#               auto boost = collision_lvec.BoostToCM();
#               return RVec<RVec<ROOT::Math::PxPyPzMVector>>{
#                 {nu_lvec, e_miss, e_miss, ROOT::Math::VectorUtil::boost(nu_lvec, boost), ROOT::Math::VectorUtil::boost(e_miss, boost)},
#                 {l_lvec + FSR_lvec, l_lvec, l_lvec + FSR_lvec, ROOT::Math::VectorUtil::boost(l_lvec, boost), ROOT::Math::VectorUtil::boost(l_lvec + FSR_lvec, boost)},
#                 {hadronic_lvec, hadronic_lvec, hadronic_lvec, ROOT::Math::VectorUtil::boost(hadronic_lvec, boost), ROOT::Math::VectorUtil::boost(hadronic_lvec, boost)}
#               };
# """, ["FSR", "emiss", "FSR_emiss", "boosted", "boosted_FSR_emiss"], "initial_lvecs")
# # FIXME needs to be commented out when not using rescanned file
# df = df.Vary("weight", "weight * alt_weight", ["g1z_up"], "aTGC")

In [10]:
# unboost crossing angle (as a variation?)
# reconstruct neutrino: missing E, w/o ovl, w/ overlay
# ISR? crossing angle, sqrt(s)
# build MC and reconstructed W, reconstructed W will vary
df = df.Define("sqrt_s_E", "Parameters.get<float>(\"Energy\").value()")
# XXX: ignoring the electron mass here
df = df.Define("sqrt_s_lvec", f"ROOT::Math::PxPyPzEVector(sqrt_s_E * sin({x_angle}/2), 0, sqrt_s_E * cos({x_angle}/2), sqrt_s_E)")
df = df.Define("visible_lvec", "e_lvec + isr1_lvec + isr2_lvec + ovl_lvec + had_sum_lvec")
# TODO: add 3 variations: no removal, naive removal (Andre), cheated removal
# Andre's cut: log_10 Pt > 0.2 sin theta + something
df = df.Define("identified_ovl_lvec", "ROOT::Math::PxPyPzEVector()")
# TODO variations: none, fitted/reconstructed, cheated ISR goes here
df = df.Define("identified_isr_lvec", "ROOT::Math::PxPyPzEVector()")
df = df.Define("visible_corrected_lvec", "visible_lvec - identified_ovl_lvec - identified_isr_lvec")

df = df.Define("missing_E_lvec", "sqrt_s_lvec - visible_corrected_lvec")
df = df.Define("reco_nu_lvec", "missing_E_lvec")

df = df.Define("W_lep_lvec", "e_lvec + reco_nu_lvec")
df = df.Define("W_had_lvec", "visible_corrected_lvec")